In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.naive_bayes import GaussianNB

from sklearn.cross_validation import train_test_split

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import tensorflow as tf

#tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here


In [3]:
# Parameters# Param 
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

In [5]:
df = pd.read_csv('../data/final_data/entrenamiento_postulaciones_vistas15.csv', encoding='utf-8')
df.head()

,idaviso,esta_online,idtipo_de_trabajo,idnivel_laboral,idnombre_area,idpostulante,month,day,dayofweek,hour,valor_educacion,sexo,edad,idnombre_area_pref,idnivel_laboral_pref,idtipo_de_trabajo_pref,se_postulo,idpostulante_int
0,1112383482,1,0,0,15,wVkd6BV,4,3,1,2,2048.0,100,26,-1,-1,-1,0,441089
1,1112483362,0,0,0,142,ZWWD4Y,4,24,1,16,1024.0,100,34,142,0,0,0,302383
2,1112366913,1,0,0,182,X9xWBKJ,3,9,4,12,64.0,-100,35,182,0,0,1,272749
3,1112406944,0,0,2,142,5mdE4wz,3,21,2,14,64.0,-100,26,142,0,0,1,60557
4,1112263557,0,0,3,168,ZxL08R,1,16,1,17,1024.0,-100,32,6,0,0,1,305463


In [6]:
df.shape

(9940108, 18)

In [ ]:
df = pd.DataFrame()
for chunk in pd.read_csv('../data/final_data/entrenamiento_postulaciones_vistas4.csv', header=None, chunksize=100000):
    df = pd.concat([df, chunk], ignore_index=True)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.shape

In [ ]:
# Network Parameters
n_hidden_1 = 41 # 1st layer number of neurons
n_hidden_2 = 80 # 2nd layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

In [ ]:
df.info(memory_usage='deep')

In [4]:
from __future__ import print_function

import numpy as np
import tensorflow as tf


In [11]:
cols = df.head(0)
cols.drop(['se_postulo', 'idaviso', 'idpostulante'],inplace=True, axis=1)
cols = cols.columns

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [29]:
X = np.array(df.loc[:,cols])
y = np.array(df['se_postulo']) 

In [16]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [15]:
!pip3 install keras

    100% |████████████████████████████████| 307kB 3.6MB/s ta 0:00:01
    100% |████████████████████████████████| 2.8MB 1.8MB/s ta 0:00:01
    100% |████████████████████████████████| 256kB 2.6MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 2.4MB/s ta 0:00:01
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directory: /home/ariel/.cache/pip/wheels/03/05/65/bdc14f2c6e09e82ae3e0f13d021e1b6b2481437ea2f207df3f
Successfully built pyyaml
You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [17]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [32]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(15, input_dim=15, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [33]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

In [34]:
print(estimator)

In [35]:
X.size

149101620

In [36]:
y.size

9940108

In [ ]:
results = cross_val_score(estimator, X, y, cv=kfold)

In [ ]:
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
# smaller model
def create_smaller():
	# create model
	model = Sequential()
	model.add(Dense(10, input_dim=15, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
# larger model
def create_larger():
	# create model
	model = Sequential()
	model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
	model.add(Dense(30, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))